<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


## Leveraging Apache Spark for Smart Building HVAC Monitoring

**Estimated time needed: 30 minutes**

### Objectives

After completing this lab, you will be able to:

- Explain the distributed architecture of Spark in the context of smart building monitoring
- Simulate real-time sensor data for HVAC systems in a building
- Perform SQL queries to detect critical environmental conditions and calculate average readings
- Determine the aggregated results to the console for immediate insights into room conditions


## Background
Smart Building Solutions, Inc. specializes in optimizing HVAC (heating, ventilation, and air conditioning) systems to enhance comfort and energy efficiency in commercial buildings. By monitoring temperature and humidity levels in real-time across various rooms, the company aims to ensure optimal indoor conditions and preemptively address potential HVAC issues.

With a continuous influx of sensor data, Smart Building Solutions needs to process and analyze this data in real-time to maintain the quality of the indoor environment.

## Data set description
The simulated data set comprises:

`room_id`: Unique identifier for each room (e.g., R001, R002).

`temperature`: Current temperature reading from the sensor (in °C).

`humidity`: Current humidity level reading from the sensor (in %).

`timestamp`: Time when the reading was recorded (automatically generated by Spark).
The data is generated at a rate of 5 rows per second, simulating multiple rooms with various environmental conditions.


## Challenges
Monitoring indoor environmental conditions poses several challenges:

**High data velocity**: Continuous data from multiple sensors can overwhelm traditional systems.

**Need for immediate alerts**: Delays in identifying critical conditions can lead to discomfort or system inefficiencies.

**Need for data aggregation and analysis**: Efficiently aggregating and analyzing real-time data for proactive maintenance and optimization is essential.

## Apache Spark with structured streaming
To address these challenges, Apache Spark is employed for its powerful distributed computing capabilities, enabling real-time data processing and analytics.


In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [2]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

#import functions/Classes for sparkml

from pyspark.ml.clustering import KMeans


from pyspark.sql import SparkSession


### Set up the Spark session:


In [3]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Smart Building HVAC Monitoring") \
    .getOrCreate()


24/10/15 18:34:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Simulate sensor data:

Use Spark’s rate source to generate continuous readings from multiple rooms.


In [5]:
from pyspark.sql.functions import expr, rand,when

# Simulate sensor data with room IDs and readings
sensor_data = spark.readStream.format("rate").option("rowsPerSecond", 5).load() \
    .withColumn("room_id", expr("CAST(value % 10 AS STRING)")) \
    .withColumn("temperature", when(expr("value % 10 == 0"), 15)  # Set temperature to 15 for one specific record
                .otherwise(20 + rand() * 25)) \
    .withColumn("humidity", expr("40 + rand() * 30"))

### Create a temporary SQL view:

Create temporary SQL view to perform SQL queries on the streaming data.


In [6]:
# Create a temporary SQL view for the sensor data
sensor_data.createOrReplaceTempView("sensor_table")


### Define SQL queries for aggregation and analysis:

* **Critical temperature query**: Detect rooms with critical temperature levels
* **Average readings query**: Calculate average readings over a 1-minute window
* **Attention needed query**: Identify rooms that need immediate attention based on humidity levels


In [7]:
# SQL Query to detect rooms with critical temperatures
critical_temperature_query = """
    SELECT 
        room_id, 
        temperature, 
        humidity, 
        timestamp 
    FROM sensor_table 
    WHERE temperature < 18 OR temperature > 60
"""

# SQL Query to calculate average readings over a 1-minute window
average_readings_query = """
    SELECT 
        room_id, 
        AVG(temperature) AS avg_temperature, 
        AVG(humidity) AS avg_humidity, 
        window.start AS window_start 
    FROM sensor_table
    GROUP BY room_id, window(timestamp, '1 minute')
"""

# SQL Query to find rooms that need immediate attention based on humidity
attention_needed_query = """
    SELECT 
        room_id, 
        COUNT(*) AS critical_readings 
    FROM sensor_table 
    WHERE humidity < 45 OR humidity > 75
    GROUP BY room_id
"""


### Execute the SQL queries:

Execute each SQL query to create streaming DataFrames.


In [8]:
# Execute the critical temperature query
critical_temperatures_stream = spark.sql(critical_temperature_query)


# Execute the average readings query
average_readings_stream = spark.sql(average_readings_query)

# Execute the attention needed query
attention_needed_stream = spark.sql(attention_needed_query)






### Output the results to the console:

Display the results from each query in real-time.


In [9]:
# Output the results to the console for all queries
critical_query = critical_temperatures_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .queryName("Critical Temperatures") \
    .start()

average_query = average_readings_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .queryName("Average Readings") \
    .start()

attention_query = attention_needed_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .queryName("Attention Needed") \
    .start()



-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-----------+--------+---------+
|room_id|temperature|humidity|timestamp|
+-------+-----------+--------+---------+
+-------+-----------+--------+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------+-----------+-----------------+--------------------+
|room_id|temperature|         humidity|           timestamp|
+-------+-----------+-----------------+--------------------+
|      0|       15.0| 52.6332847896147|2024-10-15 18:39:...|
|      0|       15.0|52.44366814901971|2024-10-15 18:39:...|
|      0|       15.0| 64.1879841461621|2024-10-15 18:39:...|
|      0|       15.0|48.56113839067988|2024-10-15 18:39:...|
+-------+-----------+-----------------+--------------------+



-------------------------------------------
Batch: 0
-------------------------------------------
+-------+---------------+------------+------------+
|room_id|avg_temperature|avg_humidity|window_start|
+-------+---------------+------------+------------+
+-------+---------------+------------+------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 55.29854763441461|2024-10-15 18:39:...|
|      0|       15.0|60.159733105002964|2024-10-15 18:39:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+-------+-----------------+
+-------+-----------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 66.79601794976932|2024-10-15 18:39:...|
|      0|       15.0| 62.48507883183857|2024-10-15 18:39:...|
|      0|       15.0| 53.43464217089421|2024-10-15 18:39:...|
|      0|       15.0| 63.76421487190079|2024-10-15 18:39:...|
|      0|       15.0|63.819230615048774|2024-10-15 18:39:...|
|      0|       15.0| 46.09277138964474|2024-10-15 18:39:...|
|      0|       15.0| 60.35882633195676|2024-10-15 18:39:...|
|      0|       15.0|62.422984013077155|2024-10-15 18:39:...|
|      0|       15.0|49.772415513679064|2024-10-15 18:39:...|
|      0|       15.0| 59.31012888964979|2024-10-15 18:39:...|
|      0|       15.0| 47.95551368936084|2024-10-15 18:39:...|
|      0|       15.0|61.67828694493

...

[Stage 46:(191 + 8) / 200][Stage 47:>   (0 + 0) / 8][Stage 48:>   (0 + 0) / 8]8]

-------------------------------------------
Batch: 16
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 40.34258324526214|2024-10-15 18:42:...|
|      0|       15.0| 66.98008865680814|2024-10-15 18:42:...|
|      0|       15.0|51.859050527022646|2024-10-15 18:42:...|
|      0|       15.0|54.047416515675565|2024-10-15 18:42:...|
|      0|       15.0| 67.35930998606847|2024-10-15 18:42:...|
+-------+-----------+------------------+--------------------+



-------------------------------------------
Batch: 7
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+-------+-----------------+
|      7|               12|
|      3|               18|
|      8|               20|
|      0|               19|
|      5|               24|
|      6|               14|
|      9|               18|
|      1|               18|
|      4|               22|
|      2|               21|
+-------+-----------------+



-------------------------------------------
Batch: 17
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0|  68.7289220266812|2024-10-15 18:42:...|
|      0|       15.0|56.661215751900244|2024-10-15 18:42:...|
|      0|       15.0| 68.27130735994848|2024-10-15 18:42:...|
|      0|       15.0| 41.95483394669838|2024-10-15 18:42:...|
|      0|       15.0|  43.6098109832023|2024-10-15 18:42:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 8
-------------------------------------------
+-------+------------------+------------------+-------------------+
|room_id|   avg_temperature|      avg_humidity|       window_start|
+-------+------------------+------------------+-------------------+
|      9| 33.92193270484829| 53.9036138669

[Stage 52:(142 + 8) / 200][Stage 53:>   (0 + 0) / 8][Stage 54:>   (0 + 0) / 8]8]

### Keep the streams running:

Ensure that the streaming queries continue to run to process incoming data.


In [ ]:
# Keep the streams running

print("********Critical Temperature Values*******")
critical_query.awaitTermination()

print("********Average Readings Values********")
average_query.awaitTermination()
print("********Attention Needed Values********")
attention_query.awaitTermination()


[Stage 52:(151 + 8) / 200][Stage 53:>   (0 + 0) / 8][Stage 54:>   (0 + 0) / 8]

********Critical Temperature Values*******


-------------------------------------------
Batch: 18
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 46.57086995807084|2024-10-15 18:42:...|
|      0|       15.0| 54.95766545607238|2024-10-15 18:43:...|
|      0|       15.0| 64.86950233854219|2024-10-15 18:42:...|
|      0|       15.0|42.306939281774945|2024-10-15 18:43:...|
|      0|       15.0|54.735186930590594|2024-10-15 18:42:...|
|      0|       15.0| 68.13880406938013|2024-10-15 18:43:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 8
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+-------+-----------------+
|      7|               14|
|      3|               19|
|      8|               24|
|      0|       

[Stage 55:(190 + 8) / 200][Stage 56:>   (0 + 0) / 8][Stage 57:>   (0 + 0) / 8]8]

-------------------------------------------
Batch: 19
-------------------------------------------


...

-------------------------------------------
Batch: 138
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0|48.819899416705375|2024-10-15 19:05:...|
|      0|       15.0|62.083554779017604|2024-10-15 19:06:...|
|      0|       15.0|53.434889119362765|2024-10-15 19:05:...|
|      0|       15.0|43.014121930889914|2024-10-15 19:06:...|
|      0|       15.0| 55.43320394585207|2024-10-15 19:05:...|
|      0|       15.0|  62.8639862644187|2024-10-15 19:06:...|
|      0|       15.0| 60.15142857257695|2024-10-15 19:06:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 68
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+-------+-----------------+
|      7|              125|
|      3

-------------------------------------------
Batch: 69
-------------------------------------------
-------------------------------------------
Batch: 139
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 44.56674613137801|2024-10-15 19:06:...|
|      0|       15.0|62.040793232612835|2024-10-15 19:06:...|
|      0|       15.0| 41.82711777464038|2024-10-15 19:06:...|
|      0|       15.0|  67.7338923913976|2024-10-15 19:06:...|
|      0|       15.0| 68.27542667526849|2024-10-15 19:06:...|
|      0|       15.0| 49.00272903256473|2024-10-15 19:06:...|
+-------+-----------+------------------+--------------------+

+-------+------------------+------------------+-------------------+
|room_id|   avg_temperature|      avg_humidity|       window_start|
+-------+------------------+------------------

[Stage 418:(191 + 8) / 200][Stage 419:>  (0 + 0) / 8][Stage 420:>  (0 + 0) / 8]]

-------------------------------------------
Batch: 140
-------------------------------------------


+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0|55.253511382624595|2024-10-15 19:06:...|
|      0|       15.0| 59.56352096716981|2024-10-15 19:06:...|
|      0|       15.0| 68.65751824846707|2024-10-15 19:06:...|
|      0|       15.0|43.886910411940285|2024-10-15 19:06:...|
|      0|       15.0| 67.73439192079644|2024-10-15 19:06:...|
|      0|       15.0|52.401327171943045|2024-10-15 19:06:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 69
-------------------------------------------


### Conclusion
In this lab, you explored the use of Apache Spark in smart building monitoring, particularly focusing on HVAC (heating, ventilation, and air conditioning) systems. You now understand the Spark's distributed architecture. You also understand how to simulate real-time sensor data for temperature and humidity, execute SQL queries to identify critical environmental conditions, and output aggregated results for immediate insights.


## Author(s)

Lakshmi Holla

## Other Contributors
Malika Singla
